# Dates
The Dates module provides two types for working with dates: Date and DateTime, representing day and millisecond precision, respectively; both are subtypes of the abstract TimeType. The motivation for distinct types is simple: some operations are much simpler, both in terms of code and mental reasoning, when the complexities of greater precision don't have to be dealt with. For example, since the Date type only resolves to the precision of a single date (i.e. no hours, minutes, or seconds), normal considerations for time zones, daylight savings/summer time, and leap seconds are unnecessary and avoided.


## Constructors
`Date` and `DateTime` types can be constructed by `integer` or `Period` types, by parsing, or through adjusters (more on those later):

In [4]:
using Dates
DateTime(2013)


2013-01-01T00:00:00

In [5]:
DateTime(2013,7,1,12,30)

2013-07-01T12:30:00

In [6]:
Date(2013)

2013-01-01

In [7]:
Date(2013,7,1)

2013-07-01

In [9]:
Date(Dates.Year(2013),Dates.Month(7),Dates.Day(1))

2013-07-01


`Date` or `DateTime` parsing is accomplished by the use of format strings. Format strings work by the notion of defining **delimited** or **fixed-width** "slots" that contain a period to parse and passing the text to parse and format string to a Date or DateTime constructor, of the form `Date("2015-01-01",dateformat"y-m-d")` or `DateTime("20150101",dateformat"yyyymmdd")`.

Delimited slots are marked by specifying the delimiter the parser should expect between two subsequent periods; so `"y-m-d"` lets the parser know that between the first and second slots in a date string like `"2014-07-16"`, it should find the - character. The y, m, and d characters let the parser know which periods to parse in each slot.

In [10]:
for i = 1:10^5
    Date("2015-01-01", dateformat"y-m-d")
end

In [11]:
df = DateFormat("y-m-d");
dt = Date("2015-01-01",df)

2015-01-01

In [12]:
dt2 = Date("2015-01-02",df)

2015-01-02

In [13]:
years = ["2015", "2016"];
Date.(years, DateFormat("yyyy"))

2-element Vector{Date}:
 2015-01-01
 2016-01-01

## Durations/Comparisons


In [15]:
dt = Date(2012,2,29)
dt2 = Date(2000,2,1)

dump(dt)


Date
  instant: Dates.UTInstant{Day}
    periods: Day
      value: Int64 734562


In [16]:
dump(dt2)

Date
  instant: Dates.UTInstant{Day}
    periods: Day
      value: Int64 730151


In [17]:
dt > dt2

true

In [18]:
dt + dt2

LoadError: MethodError: no method matching +(::Date, ::Date)
[0mClosest candidates are:
[0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:560
[0m  +(::TimeType, [91m::StridedArray{var"#s814", N} where {var"#s814"<:Union{Dates.CompoundPeriod, Period}, N}[39m) at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Dates/src/deprecated.jl:18
[0m  +([91m::StridedArray{var"#s814", N} where {var"#s814"<:Union{Dates.CompoundPeriod, Period}, N}[39m, ::TimeType) at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Dates/src/deprecated.jl:10
[0m  ...

In [19]:
dt - dt2

4411 days

In [20]:
dt = DateTime(2012,2,29)

2012-02-29T00:00:00

In [21]:
dt2 = DateTime(2000,2,1)

2000-02-01T00:00:00

In [22]:
dt - dt2


381110400000 milliseconds

## Accessor Functions

Because the Date and DateTime types are stored as single Int64 values, date parts or fields can be retrieved through accessor functions. The lowercase accessors return the field as an integer:

In [23]:
t = Date(2014, 1, 31)
Dates.year(t)

2014

In [24]:
Dates.month(t)

1

In [25]:
Dates.week(t)


5

In [26]:
Dates.day(t)

31

While propercase return the same value in the corresponding Period type:

In [27]:
Dates.Year(t)

2014 years

In [28]:
Dates.Day(t)

31 days

Compound methods are provided because it is more efficient to access multiple fields at the same time than individually:

In [29]:
Dates.yearmonth(t)

(2014, 1)

In [30]:
Dates.monthday(t)

(1, 31)

In [32]:
Dates.yearmonthday(t)

(2014, 1, 31)

## Query Functions

Query functions provide calendrical information about a TimeType. They include information about the day of the week:

In [33]:
t = Date(2014, 1, 31)

2014-01-31

In [34]:
Dates.dayofweek(t)

5

In [35]:
Dates.dayname(t)

"Friday"

In [36]:
Dates.dayofweekofmonth(t) # 5th Friday of January

5

In [37]:
Dates.monthname(t)

"January"

In [38]:
Dates.daysinmonth(t)

31

In [39]:
Dates.isleapyear(t)

false

In [40]:
Dates.dayofyear(t)

31

In [41]:
Dates.quarterofyear(t)

1

In [42]:
Dates.dayofquarter(t)

31

## TimeType-Period Arithmetic

In [43]:
(Date(2014,1,29)+Dates.Day(1)) + Dates.Month(1)

2014-02-28

In [44]:
(Date(2014,1,29)+Dates.Month(1)) + Dates.Day(1)

2014-03-01

In [45]:
dr = Date(2014,1,29):Day(1):Date(2014,2,3)
collect(dr)

6-element Vector{Date}:
 2014-01-29
 2014-01-30
 2014-01-31
 2014-02-01
 2014-02-02
 2014-02-03

In [46]:
dr = Date(2014,1,29):Dates.Month(1):Date(2014,07,29)

Date("2014-01-29"):Month(1):Date("2014-07-29")

In [47]:
collect(dr)

7-element Vector{Date}:
 2014-01-29
 2014-02-28
 2014-03-29
 2014-04-29
 2014-05-29
 2014-06-29
 2014-07-29

## Adjuster Functions
As convenient as date-period arithmetic is, often the kinds of calculations needed on dates take on a calendrical or temporal nature rather than a fixed number of periods. 

In [48]:
Dates.firstdayofweek(Date(2014,7,16))

2014-07-14

In [49]:
Dates.firstdayofmonth(Date(2014,7,16))

2014-07-01

In [50]:
Dates.lastdayofmonth(Date(2014,7,16)) 

2014-07-31

In [51]:
Dates.lastdayofquarter(Date(2014,7,16)) 

2014-09-30

In [52]:
istuesday = x->Dates.dayofweek(x) == Dates.Tuesday;

In [53]:
Dates.tonext(istuesday, Date(2014,7,13))  Dates.tonext(istuesday, Date(2014,7,13)) 

LoadError: syntax: extra token "Dates" after end of expression

## Rounding

Date and DateTime values can be rounded to a specified resolution (e.g., 1 month or 15 minutes) with floor, ceil, or round:

In [54]:
floor(Date(1985, 8, 16), Dates.Month)

1985-08-01

In [55]:
ceil(DateTime(2013, 2, 13, 0, 31, 20), Dates.Minute(15))

2013-02-13T00:45:00

In [56]:
round(DateTime(2016, 8, 6, 20, 15), Dates.Day)

2016-08-07T00:00:00

In [57]:
# Rounding Epoch
round(DateTime(2016, 7, 17, 11, 55), Dates.Hour(10))

2016-07-17T12:00:00